In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
import sys
sys.path.append("..")

In [4]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [5]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [6]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [7]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")
parsed_rom = rom.parse()

In [8]:
all_posns = all_positions(rooms, parsed_rom, is_global=False)
all_posns["Bomb_Torizo_Bombs"] = all_posns["Bomb_Torizo_B"]

West_Ocean 0x793fe
	 Extra: ['West_Ocean_R4', 'West_Ocean_L']
	 Missing: []
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: []


In [9]:
import omega
from omega.symbolic.fol import Context

In [10]:
#all_posns

In [11]:
#design["Rooms"]["Botwoon_Energy"]

In [12]:
#all_global_positions

In [13]:
#design["Items"], design["Bosses"]

In [14]:
room_addrs = {name: room.mem_address & 0xffff for name, room in rooms.items()}

In [15]:
def loc(room_name, node_name, when="prev"):
    #TODO: local positions
    #TODO: with room_prev and room_next
    try:
        node_loc = all_posns[f"{room_name}_{node_name}"]
    except KeyError:
        node_loc = Coord(0,0)
    rr = f"room_id_{when} = {rooms[room_name].mem_address & 0xffff}"
    xx = f"x_{when} = {int(node_loc.x)}"
    yy = f"y_{when} = {int(node_loc.y)}"
    #TODO room_id
    return " & ".join([rr, xx, yy])

def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)


def itemset_to_str(itemset):
    if len(itemset) == 0:
        return "TRUE"
    else:
        return " & ".join([f"{item}_prev = 1" for item in itemset])

def required_itemsets(itemsets):
    return "(" + " | ".join([itemset_to_str(itemset) for itemset in itemsets]) + ")"


In [16]:
from tqdm.notebook import tqdm

In [17]:
items_unchanged_def = "items_unchanged == " + " & ".join([f"{i}_prev = {i}_next" for i in design["Items"] | design["Bosses"]])
room_unchanged_def = "room_unchanged == room_id_prev = room_id_next"

In [18]:
items_unchanged_def

'items_unchanged == B_prev = B_next & PB_prev = PB_next & SPB_prev = SPB_next & S_prev = S_next & M_prev = M_next & G_prev = G_next & SA_prev = SA_next & V_prev = V_next & GS_prev = GS_next & SB_prev = SB_next & HJ_prev = HJ_next & MB_prev = MB_next & CB_prev = CB_next & WB_prev = WB_next & E_prev = E_next & PLB_prev = PLB_next & IB_prev = IB_next & SJ_prev = SJ_next & Spazer_prev = Spazer_next & RT_prev = RT_next & XR_prev = XR_next & Drain_prev = Drain_next & Shaktool_prev = Shaktool_next & START_prev = START_next & Statues_prev = Statues_next & Kraid_prev = Kraid_next & Phantoon_prev = Phantoon_next & Draygon_prev = Draygon_next & Ridley_prev = Ridley_next & Mother_Brain_prev = Mother_Brain_next & Botwoon_prev = Botwoon_next & Spore_Spawn_prev = Spore_Spawn_next & Golden_Torizo_prev = Golden_Torizo_next & Bomb_Torizo_prev = Bomb_Torizo_next & Crocomire_prev = Crocomire_next'

In [19]:
from omega.logic.lexyacc import Parser

In [20]:
p = Parser()

In [21]:
p.parse("LET x == 7 IN y")

Operator('LET', [Binary('==', Terminal('x', 'opname'), Num('7', 'num'))], Var('y', 'var'))

In [22]:
from bdds.bdd_core import *

In [23]:
from functools import reduce

In [24]:
# Second attempt

In [25]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [26]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [27]:
# Only encode node_id and items
def mk_context_id():
    context = Context()
    context.declare(
        node_id_prev = (0,len(node_ids)),
        node_id_next = (0,len(node_ids)),
        node_id_goal = (0,len(node_ids)),
        node_id_temp = (0,len(node_ids)),
    )
    # Try to get a favorable variable ordering
    shared_dict = {}
    for i in item_mapping:
        shared_dict[f"{i}_prev"] = (0,1)
        shared_dict[f"{i}_next"] = (0,1)
        shared_dict[f"{i}_goal"] = (0,1)
        shared_dict[f"{i}_temp"] = (0,1)
    context.declare(**shared_dict)
    context.define(mk_items_unchanged())
    return context

In [28]:
context_id = mk_context_id()

In [29]:
context_id.bdd.configure(reordering=False)

{'reordering': True,
 'garbage_collection': True,
 'max_memory': 18446744073709551615,
 'loose_up_to': 6710886,
 'max_cache_soft': 189440,
 'max_cache_hard': 4294967295,
 'min_hit': 30,
 'max_growth': 1.2,
 'max_swaps': 2000000,
 'max_vars': 1000}

In [30]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context_id.false)

In [31]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return f"node_id_{when} = {node_id}"

In [32]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions()
        links.append(context_id.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name])
            links.append(context_id.add_expr(s, with_ops=True))
    for node_name, door in room['Doors'].items():
        a1 = room_addrs[room_name]
        a2 = room_addrs[door["Room"]]
        d = loc_id(room_name, node_name) + " & " + loc_id(door['Room'], door['Node'], when="next") + " & " + item_transitions()
        door_bdds.append(context_id.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions()
            links.append(context_id.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:15<00:00, 16.41it/s]


In [33]:
doors_bdd = combine_bdd_group(door_bdds)
rooms_bdd = combine_bdd_group(room_bdds)
all_bdd = doors_bdd | rooms_bdd

In [34]:
all_bdd.dag_size # 11K

13069

In [35]:
# Have to refine some stuff from bdd_core for the new context
prev_to_temp = mk_replace(context_id, "prev", "temp")
next_to_temp = mk_replace(context_id, "next", "temp")
goal_to_next = mk_replace(context_id, "goal", "next")
prev_to_next = mk_replace(context_id, "prev", "next")
next_to_prev = mk_replace(context_id, "next", "prev")
temps = [k for k in context_id.vars.keys() if k.endswith("_temp")]
nexts = [k for k in context_id.vars.keys() if k.endswith("_next")]
prevs = [k for k in context_id.vars.keys() if k.endswith("_prev")]
goals = [k for k in context_id.vars.keys() if k.endswith("_goal")]

def mk_goal_satisfied_id(context):
    # Expression that holds when the goal is satisfied (for all possible goals)
    goal_satisfied = context.true
    for v1, v2 in zip(nexts, goals):
        sat = context.add_expr(f"{v1} = {v2}")
        #print(sat.dag_size)
        #print(goal_satisfied.dag_size)
        goal_satisfied &= sat
    return goal_satisfied

def mk_policy_id(trans_norule, context, goal_bdd=None):
    n = 0
    # prev, next, goal
    policy = context.false
    covered = mk_goal_satisfied_id(context)
    if goal_bdd:
        covered &= goal_bdd
    covered_last = context.false
    while covered != covered_last:
        # States with edges into a covered state
        covered_last = covered
        covered_temp = context.exist(nexts, trans_norule & covered)
        covered_temp = context.let(prev_to_next, covered_temp)
        # policy(prev, next, goal) = trans(prev, next) & covered(next, goal) & ~covered(prev, goal)
        covered_prev_goal = context.let(next_to_prev, covered)
        policy |= trans_norule & covered & ~covered_prev_goal
        # Find a state in covered that the fringe state transitions to
        covered |= covered_temp
        print(n, covered_temp.dag_size, covered.dag_size, policy.dag_size)
        n += 1
    return policy

def mk_policy_id2(trans_norule_pn, context, goal_bdd=None):
    n = 0
    # prev, next, goal
    policy_png = context.false
    covered_ng = mk_goal_satisfied_id(context)
    if goal_bdd:
        covered_ng &= goal_bdd
    covered_last_ng = context.false
    while covered_ng != covered_last_ng:
        # States with edges into a covered state
        covered_last_ng = covered_ng
        fringe_pg = context.exist(nexts, trans_norule_pn & covered_ng)
        fringe_ng = context.let(prev_to_next, fringe_pg) & ~covered_ng
        # policy(prev, next, goal) = trans(prev, next) & fringe(next, goal) & ~covered(prev, goal)
        policy_png |= trans_norule_pn & fringe_ng
        # Find a state in covered that the fringe state transitions to
        covered_ng |= fringe_ng
        print(n, fringe_ng.dag_size, covered_ng.dag_size, policy_png.dag_size)
        n += 1
    return policy_png

In [36]:
from data_types import item_set

In [37]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = f"{i}_{when} = 1"
        else:
            clause = f"{i}_{when} = 0"
        clauses.append(clause)
    return " & ".join(clauses)

In [45]:
def get_reachable_id(trans_norule, context, start_bdd):
    n = 0
    covered_p = start_bdd
    covered_last_p = context.false
    while covered_p != covered_last_p:
        covered_last_p = covered_p
        fringe_n = context.exist(prevs, covered_p & trans_norule)
        covered_p |= context.let(next_to_prev, fringe_n)
        print(n, covered_p.dag_size)
        n+=1
    return covered_p

In [46]:
all_bdd_unchanged = all_bdd & context_id.add_expr("items_unchanged", with_ops=True)

In [47]:
node_ids["Landing_Site_L2"]

3

In [48]:
mk_itemset_expr(item_set.ItemSet())

'B_prev = 0 & PB_prev = 0 & SPB_prev = 0 & S_prev = 0 & M_prev = 0 & G_prev = 0 & SA_prev = 0 & V_prev = 0 & GS_prev = 0 & SB_prev = 0 & HJ_prev = 0 & MB_prev = 0 & CB_prev = 0 & WB_prev = 0 & E_prev = 0 & PLB_prev = 0 & Spazer_prev = 0 & RT_prev = 0 & XR_prev = 0 & IB_prev = 0 & SJ_prev = 0 & Kraid_prev = 0 & Phantoon_prev = 0 & Draygon_prev = 0 & Ridley_prev = 0 & Botwoon_prev = 0 & Spore_Spawn_prev = 0 & Golden_Torizo_prev = 0 & Bomb_Torizo_prev = 0 & Mother_Brain_prev = 0 & Crocomire_prev = 0 & Ceres_Ridley_prev = 0 & Drain_prev = 0 & Shaktool_prev = 0 & START_prev = 0 & Statues_prev = 0'

In [51]:
start_bdd = context_id.add_expr("node_id_prev = 3 & " + mk_itemset_expr(item_set.ItemSet()))

In [52]:
all_reachable_p = get_reachable_id(all_bdd, context_id, start_bdd)

0 50
1 51
2 57
3 59
4 61
5 62
6 61
7 61
8 65
9 67
10 105
11 105
12 105
13 121
14 125
15 123
16 134
17 129
18 123
19 156
20 161
21 140
22 144
23 143
24 143
25 144
26 143
27 142
28 143
29 151
30 144
31 135
32 138
33 138
34 143
35 145
36 143
37 147
38 147
39 160
40 163
41 160
42 178
43 183
44 187
45 195
46 204
47 200
48 192
49 195
50 199
51 188
52 192
53 193
54 208
55 211
56 231
57 225
58 250
59 281
60 287
61 303
62 313
63 323
64 315
65 320
66 323
67 337
68 348
69 360
70 365
71 388
72 385
73 386
74 383
75 373
76 367
77 359
78 363
79 368
80 403
81 372
82 396
83 388
84 400
85 414
86 416
87 412
88 418
89 410
90 431
91 419
92 429
93 425
94 436
95 426
96 455
97 429
98 464
99 453
100 492
101 500
102 554
103 557
104 527
105 549
106 600
107 621
108 644
109 665
110 675
111 674
112 694
113 702
114 716
115 713
116 702
117 706
118 656
119 679
120 734
121 725
122 712
123 703
124 676
125 700
126 674
127 684
128 647
129 636
130 611
131 591
132 603
133 597
134 615
135 604
136 600
137 599
138 582
139 590


In [53]:
wincon_p = context_id.add_expr("node_id_prev = 2 & Mother_Brain_prev = 1")

In [54]:
all_reachable_p.count()

1779252.0

In [55]:
(wincon_p & all_reachable_p).count()

12.0

In [56]:
feasible_trans = all_bdd & all_reachable_p

In [57]:
feasible_trans.dag_size / all_bdd.dag_size

1.0795776264442574

In [58]:
#context_id.vars

In [59]:
wincon = context_id.add_expr("node_id_goal = 2 & Mother_Brain_goal = 1")

In [60]:
# pick a concrete wincon
#wincon = context_id

In [61]:
# Use forward model to pick a feasible endpoint?
wincon = context_id.add_expr("node_id_goal = 2 & M_goal = 1")

In [62]:
import math

In [63]:
#TODO Bound the graph using forward symbolic search...?
# try no goal condition?
math.log(all_bdd.count(), 2)

47.169122498329315

In [64]:
#TODOs
# Goal BDD x
#   More restrictive goal_bdd using pick w/ transitive closure
#   Forward & backward feasibility restrictions
# ~covered_prev_goal earlier
# Shattergate
all_policy = mk_policy_id2(feasible_trans, context_id, wincon)

0 1072 1193 1716
1 839 1197 1767
2 840 1199 1833
3 827 1210 2120
4 1188 1318 2555
5 1419 1562 3032
6 1443 1594 3087
7 1339 1607 3330
8 1460 1723 3419
9 1466 1742 4039
10 1786 2061 5521
11 2296 2689 6746
12 2583 3210 7803
13 3143 3799 9034
14 3312 3829 9952
15 3428 4127 10576
16 3445 4334 11731
17 3859 4785 12421
18 4143 5043 13300
19 4156 5623 14296
20 4165 5537 14569
21 4327 6288 16183
22 4879 6819 16750
23 4943 6887 17439
24 4814 7389 18035
25 4520 7367 18270
26 4750 7856 19682
27 4952 8445 21330
28 5327 8994 22473
29 5552 9463 23735
30 6291 10226 25194
31 6594 10792 27204
32 7441 11393 28566
33 7656 12597 32632
34 9145 14083 33950
35 9685 15064 36568
36 10096 15470 36877
37 9807 16626 41040
38 11327 17600 40664
39 11214 18175 43436
40 12274 18623 43649
41 12199 20768 49579
42 14600 22308 50406
43 14759 23071 54305
44 16708 24882 55183
45 17082 26866 61425
46 19296 27674 60533
47 18696 29597 67233
48 22244 31875 67662
49 22387 34243 74808
50 25315 35870 76048
51 24903 38735 84033
52 

In [ ]:
#TODO: re-implement search and figure out how long it takes to search.

In [94]:
context_id.bdd.dump("../output/policy_id.json", [all_policy])

In [95]:
task_expr = context_id.add_expr("Mother_Brain_prev < Mother_Brain_goal & node_id_prev = 3 & node_id_goal = 3")
# Concrete task
task = context_id.exist(nexts, all_policy & task_expr).pick()
task_bdd = context_id.bdd.cube(task)

In [108]:
def mk_goal_satisfied_id(context):
    # Expression that holds when the goal is satisfied (for all possible goals)
    goal_satisfied = context.true
    for v1, v2 in zip(nexts, goals):
        sat = context.add_expr(f"{v1} = {v2}")
        #print(sat.dag_size)
        #print(goal_satisfied.dag_size)
        goal_satisfied &= sat
    return goal_satisfied


In [109]:
goal_satisfied_id = mk_goal_satisfied_id(context_id)

In [129]:
# task_bdd is built by first creating a task expr, then existing off the nexts, then picking and cubing it.
def get_path_concrete_id(policy, task_bdd, context):
    # Get a concrete path
    current = context.pick(task_bdd)
    path = [current]
    n = 0
    while True:
        print(n, current["node_id_prev"])
        #print(current["node_id_prev"])
        #print(current)
        n += 1
        #current_bdd = context.bdd.cube(current)
        current_bdd = context.assign_from(current)
        next_temp = context.exist(prevs, current_bdd & policy)
        advice = context.let(next_to_prev, next_temp)
        #print(advice.count())
        current = context.pick(advice)
        # Check whether the goal is satisfied in the state given by the advice.
        if (next_temp & goal_satisfied_id).pick():
            break
        path.append(current)
    return path

In [132]:
current = context_id.pick(task_bdd)

In [134]:
cbdd = context_id.assign_from(current)

In [141]:
next_temp = context_id.exist(prevs, all_policy & cbdd)

In [143]:
advice = context_id.let(next_to_prev, next_temp)

In [146]:
current = context_id.pick(advice)

In [147]:
cbdd = context_id.assign_from(current)

In [148]:
next_temp = context_id.exist(prevs, all_policy & cbdd)

In [150]:
advice = context_id.let(next_to_prev, next_temp)

In [151]:
context_id.pick(advice)

{'node_id_prev': 0,
 'node_id_goal': 2,
 'B_prev': 0,
 'B_goal': 1,
 'PB_prev': 0,
 'PB_goal': 1,
 'SPB_prev': 0,
 'SPB_goal': 0,
 'S_prev': 0,
 'S_goal': 1,
 'M_prev': 0,
 'M_goal': 1,
 'G_prev': 0,
 'G_goal': 1,
 'SA_prev': 0,
 'SA_goal': 0,
 'V_prev': 0,
 'V_goal': 1,
 'GS_prev': 0,
 'GS_goal': 1,
 'SB_prev': 0,
 'SB_goal': 1,
 'HJ_prev': 0,
 'HJ_goal': 1,
 'MB_prev': 0,
 'MB_goal': 1,
 'CB_prev': 0,
 'CB_goal': 1,
 'WB_prev': 0,
 'WB_goal': 0,
 'E_prev': 0,
 'E_goal': 1,
 'PLB_prev': 0,
 'PLB_goal': 1,
 'Spazer_prev': 0,
 'Spazer_goal': 0,
 'RT_prev': 0,
 'RT_goal': 0,
 'XR_prev': 0,
 'XR_goal': 0,
 'IB_prev': 0,
 'IB_goal': 1,
 'SJ_prev': 0,
 'SJ_goal': 1,
 'Kraid_prev': 0,
 'Kraid_goal': 1,
 'Phantoon_prev': 0,
 'Phantoon_goal': 1,
 'Draygon_prev': 0,
 'Draygon_goal': 1,
 'Ridley_prev': 0,
 'Ridley_goal': 1,
 'Botwoon_prev': 0,
 'Botwoon_goal': 0,
 'Spore_Spawn_prev': 0,
 'Spore_Spawn_goal': 0,
 'Golden_Torizo_prev': 0,
 'Golden_Torizo_goal': 0,
 'Bomb_Torizo_prev': 0,
 'Bomb_Tor

In [130]:
get_path_concrete_id(all_policy, task_bdd, context_id)

0 2
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
180 0
181 0
182 0
183 0
184 0


KeyboardInterrupt: 

In [86]:
all_policy2 = mk_policy_id2(feasible_trans, context_id)

0 15873 17663 45546
1 19550 26609 69287
2 20833 28816 73256
3 22425 31908 80510
4 25258 35914 90141
5 28020 39735 97390
6 30600 43279 105181
7 33089 48172 115052
8 36159 52555 123808
9 39442 57622 134243
10 43101 62772 144930
11 48466 70461 160664
12 52834 77639 177226
13 58929 85573 197007
14 66926 96537 219833
15 75544 108077 244257
16 83264 119254 269222
17 92544 132864 298398
18 104951 150456 333680
19 113611 166671 370707
20 130464 189709 416987
21 143131 210443 464001
22 162315 237982 515914
23 173944 266180 578961
24 195600 299508 643323
25 214816 336218 722559
26 244600 380521 810380
27 268748 430802 914692
28 303565 482151 1014581
29 328830 535831 1128092
30 375596 599425 1253455
31 409457 675122 1409207
32 467026 754803 1559192


KeyboardInterrupt: 

In [ ]:
# Here's how to run X analyses
# Iterative squaring

In [ ]:
# First attempt

In [23]:
context = mk_context()

In [24]:
context.op_bdd

{'items_unchanged': Function (DdNode) with var index: 203, ref count: 1, int repr: 73734274}

In [25]:
context.define(room_unchanged_def)

In [26]:
design["Rooms"]["Landing_Site"]["Edges"]["Ship"]

[{'Terminal': 'L2', 'Requirements': [[]]}]

In [27]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context.false)

In [29]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions() + " & room_unchanged"
        links.append(context.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            s = loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]) + " & room_unchanged"
            links.append(context.add_expr(s, with_ops=True))
    for node_name, door in room['Doors'].items():
        a1 = room_addrs[room_name]
        a2 = room_addrs[door["Room"]]
        d = loc(room_name, node_name) + " & " + loc(door['Room'], door['Node'], when="next") + " & " + item_transitions() + f" & room_id_prev = {a1} & room_id_next = {a2}"
        door_bdds.append(context.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc(room_name, node_name) + " & " + loc(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions() + " & room_unchanged"
            links.append(context.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:39<00:00,  6.40it/s]


In [30]:
#design["Rooms"]["Bomb_Torizo"]["Drops"]

In [31]:
#(context.add_expr(item_transitions(design["Rooms"]["Bomb_Torizo"]["Drops"]["Bombs"])) & context.add_expr("B_prev < B_next")).count()

In [32]:
room_name = "Bomb_Torizo"
node_name = "Bombs"
room = design["Rooms"][room_name]
s = loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]) + " & room_unchanged"

In [33]:
s

'room_id_prev = 38916 & x_prev = 12 & y_prev = 10 & room_id_next = 38916 & x_next = 12 & y_next = 10 & B_prev < B_next & PB_prev = PB_next & SPB_prev = SPB_next & S_prev = S_next & M_prev = M_next & G_prev = G_next & SA_prev = SA_next & V_prev = V_next & GS_prev = GS_next & SB_prev = SB_next & HJ_prev = HJ_next & MB_prev = MB_next & CB_prev = CB_next & WB_prev = WB_next & E_prev = E_next & PLB_prev = PLB_next & IB_prev = IB_next & SJ_prev = SJ_next & Spazer_prev = Spazer_next & RT_prev = RT_next & XR_prev = XR_next & Drain_prev = Drain_next & Shaktool_prev = Shaktool_next & START_prev = START_next & Statues_prev = Statues_next & Kraid_prev = Kraid_next & Phantoon_prev = Phantoon_next & Draygon_prev = Draygon_next & Ridley_prev = Ridley_next & Mother_Brain_prev = Mother_Brain_next & Botwoon_prev = Botwoon_next & Spore_Spawn_prev = Spore_Spawn_next & Golden_Torizo_prev = Golden_Torizo_next & Bomb_Torizo_prev = Bomb_Torizo_next & Crocomire_prev = Crocomire_next & room_unchanged'

In [35]:
doors_bdd = combine_bdd_group(door_bdds)

In [36]:
rooms_bdd = combine_bdd_group(room_bdds)

In [37]:
all_bdd = doors_bdd | rooms_bdd

In [63]:
all_bdd.dag_size # 51K

51905

In [ ]:
rooms["Landing_Site"].mem_address

In [ ]:
from data_types.item_set import item_mapping

In [ ]:
no_items = " & ".join([f"{k}_prev = 0" for k in item_mapping])

In [ ]:
no_items

In [ ]:
all_bdd_pn = all_bdd

In [ ]:
start_bdd = context.add_expr(loc("Landing_Site", "L2")) & context.add_expr(no_items)

In [ ]:
current_reachable_p = start_bdd
last_reachable_p = context.false
i = 0
while current_reachable_p != last_reachable_p:
    print(i, current_reachable_p.dag_size)
    i += 1
    last_reachable_p = current_reachable_p
    current_reachable_p = current_reachable_p | context.let(next_to_prev, context.exist(prevs, current_reachable_p & all_bdd_pn))

In [ ]:
rooms["Bomb_Torizo"].mem_address & 0xffff

In [ ]:
(context.add_expr("room_id_prev = 38916") & current_reachable_p).count()

In [ ]:
n_nodes = 0
for room_name, room in design['Rooms'].items():
    n_nodes += len(room["Nodes"])
n_nodes

In [ ]:
for room_name, room in design['Rooms'].items():
    for node_name in room['Nodes']:
        if (context.add_expr(loc(room_name, node_name)) & current_reachable_p).count() == 0:
            print(f"{room_name}, {node_name} is not  reachable")

In [ ]:
for i in item_mapping:
    if (context.add_expr(f"{i}_prev = 1") & current_reachable_p).count() == 0:
        print(f"{i} not gettable")

In [ ]:
all_bdd.dag_size

In [ ]:
all_bdd.count()

In [ ]:
(all_bdd & current_reachable_p).count()

In [61]:
# Construct a node-level policy for the full game
# Unsurprisingly, this takes a while
#all_policy = mk_policy(all_bdd & current_reachable_p, context)
all_policy = mk_policy(all_bdd, context)

KeyboardInterrupt: 

In [ ]:
hex(rooms["Landing_Site"].mem_address)

In [ ]:
hex(room_addrs["Landing_Site"])

In [ ]:
len(rooms)

In [ ]:
len(doors)

In [ ]:
parsed_rom.values()

In [ ]:
rhs = [v for k,v in parsed_rom.items() if k.startswith("room_header")]

In [ ]:
sum([r.width * r.height * 256 for r in rhs])

In [ ]:
446720 / 25